In [1]:
try:
    from PIL import Image
except ImportError:
    import Image
    
import pytesseract
pytesseract.pytesseract.tesseract_cmd=r"C:\Program Files\Tesseract-OCR\tesseract.exe"
import numpy as np

def ocr_core(filename,mode=3):
    text = pytesseract.image_to_string(Image.open(filename),config=f'--psm {mode}')
    return text

def fetch_instruction(text):
    for i in text.split('\n'):
        if "take" in i.lower():
            return i.split(".")[0].lower()
    return ""
def count_digits_percent(string):
    if not string:
        return 1
    return sum([char in "0123456789" for char in string])/len(string)
image_file='static/uploads/med_label1.jpg'
print(fetch_instruction(ocr_core(image_file)))
test_sentence=ocr_core(image_file,3).split('\n')
sentences=[]
for i in test_sentence:
    if count_digits_percent(i)<0.3:
        sentences.append(i)        
print(sentences)

take 1 tablet(s) every morning
['2020 FR pees away PAM', 'loRATAdine 10MG TAB', 'TAKE 1 TABLET(S) EVERY MORNING', 'May be taken with or without food. May affect alertness. Warning - avoid', 'alecholic drinks', 'TOA PAYOH POLYCLINIC PHARMACY 821 1', ' ', '\x0c']


In [2]:
import json

with open('medicine_dict.json') as f:
    medicine_dict = json.load(f)

In [3]:
from scipy import spatial
def similarity(vector1,vector2):
    vector1=np.asarray(vector1, dtype='float64')
    vector2=np.asarray(vector2, dtype='float64')
    return 1 - spatial.distance.cosine(vector1, vector2)
def alphabet_vec(string):
    string=string.lower()
    vector=[]
    count_dict={}
    for i in string:
        if not count_dict.get(i,False):
            count_dict[i]=1
        else:
            count_dict[i]+=1
    for i in "abcdefghijklmnopqrstuvwxyz":
        vector.append(count_dict.get(i,0))
    return vector

In [4]:
import fasttext.util
#fasttext.util.download_model('en', if_exists='ignore')  # English
print("DONE")
ft = fasttext.load_model('cc.en.300.bin')

DONE


In [5]:
max_similarity=-1
for i in sentences:
    curr_vector=ft.get_word_vector(i.lower())
    curr_alpha_vector=alphabet_vec(i)
    for j in medicine_dict:
        medicine_vector=medicine_dict[j]
        medicine_alpha_vector=alphabet_vec(j)
        curr_similarity=similarity(curr_vector,medicine_vector)+similarity(curr_alpha_vector,medicine_alpha_vector)
        if curr_similarity>max_similarity:
            max_similarity=curr_similarity
            best_match=(i,j)

C:\Users\nuinui\anaconda3\lib\site-packages\scipy\spatial\distance.py:714: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


In [6]:
best_match

('loRATAdine 10MG TAB', 'Loratadine_Oral Lyophilisate Tab10mg S/F')